In [44]:
import latexp
from latexp import app
import dlsia

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from dlsia.core import helpers
from dlsia.core import train_scripts
from dlsia.core.networks import sparsenet
from dlsia.test_data.two_d import random_shapes
from dlsia.core.utils import latent_space_viewer
from dlsia.viz_tools import plots, draw_sparse_network

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import einops
import umap

from IPython.display import Image
import pacmap

In [88]:
N_train = 500
N_test  = 5000
noise_level = 0.50
Nxy = 32

train_data = random_shapes.build_random_shape_set_numpy(n_imgs=N_train,
                                                        noise_level=noise_level,
                                                        n_xy=Nxy)
test_data = random_shapes.build_random_shape_set_numpy(n_imgs=N_test,
                                                       noise_level=noise_level,
                                                       n_xy=Nxy)

test_GT = torch.Tensor(test_data["GroundTruth"]).unsqueeze(1)

In [90]:
which_one = "Noisy" #"GroundTruth" 

batch_size = 150

loader_params = {'batch_size': batch_size, 'shuffle': True} 
Ttrain_data = TensorDataset( torch.Tensor(train_data[which_one]).unsqueeze(1) ) 
train_loader = DataLoader(Ttrain_data, **loader_params)

loader_params = {'batch_size': batch_size,'shuffle': False} 
Ttest_data = TensorDataset( torch.Tensor(test_data[which_one][0:N_train]).unsqueeze(1) ) 
test_loader = DataLoader(Ttest_data, **loader_params)

demo_data = TensorDataset( torch.Tensor(test_data[which_one]).unsqueeze(1) ) 
demo_loader = DataLoader(demo_data, **loader_params)

In [106]:
autoencoders = []
for ii in range(3):
    autoencoder = sparsenet.SparseAutoEncoder(in_shape=(32, 32), 
                                              latent_shape=(4, 4), 
                                              depth=40, 
                                              dilations=[1,2,3], 
                                              hidden_channels=1, 
                                              out_channels=2, 
                                              alpha_range=(0.55, 0.75), 
                                              gamma_range=(0.50, 0.55), 
                                              max_degree=40, min_degree=1, 
                                              pIL=0.25, 
                                              pLO=0.25, 
                                              IO=False, 
                                              stride_base=2)
    pytorch_total_params = helpers.count_parameters(autoencoder)
    print( "Number of parameters:", pytorch_total_params)
    autoencoders.append(autoencoder)

Number of parameters: 45444
Number of parameters: 42326
Number of parameters: 43154


In [107]:
for autoencoder in autoencoders:
    device = helpers.get_device()
    learning_rate = 1e-3
    num_epochs=50
    criterion = nn.L1Loss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)
    rv = train_scripts.train_autoencoder(net=autoencoder.to(device),
                                         trainloader=train_loader,
                                         validationloader=test_loader,
                                         NUM_EPOCHS=num_epochs,
                                         criterion=criterion,
                                         optimizer=optimizer,
                                         device=device, 
                                         show=1)
    print("Best Performance:", rv[1]["CC validation"][rv[1]['Best model index']])        
    autoencoder = autoencoder.cpu()

Epoch 1 of 50 | Learning rate 1.000e-03
Training Loss: 5.9863e-01 | Validation Loss: 5.7442e-01
Training CC: 0.0060   Validation CC  : 0.0043 
Epoch 2 of 50 | Learning rate 1.000e-03
Training Loss: 5.6343e-01 | Validation Loss: 5.3943e-01
Training CC: 0.0042   Validation CC  : 0.0004 
Epoch 3 of 50 | Learning rate 1.000e-03
Training Loss: 5.2135e-01 | Validation Loss: 5.0594e-01
Training CC: 0.0042   Validation CC  : 0.0010 
Epoch 4 of 50 | Learning rate 1.000e-03
Training Loss: 4.9134e-01 | Validation Loss: 4.7513e-01
Training CC: 0.0062   Validation CC  : 0.0018 
Epoch 5 of 50 | Learning rate 1.000e-03
Training Loss: 4.6454e-01 | Validation Loss: 4.4614e-01
Training CC: 0.0097   Validation CC  : 0.0114 
Epoch 6 of 50 | Learning rate 1.000e-03
Training Loss: 4.2793e-01 | Validation Loss: 4.1885e-01
Training CC: 0.0242   Validation CC  : 0.0363 
Epoch 7 of 50 | Learning rate 1.000e-03
Training Loss: 4.1317e-01 | Validation Loss: 3.9346e-01
Training CC: 0.0527   Validation CC  : 0.0683 

Epoch 8 of 50 | Learning rate 1.000e-03
Training Loss: 2.8357e-01 | Validation Loss: 2.7994e-01
Training CC: 0.1996   Validation CC  : 0.2248 
Epoch 9 of 50 | Learning rate 1.000e-03
Training Loss: 2.7951e-01 | Validation Loss: 2.7394e-01
Training CC: 0.2415   Validation CC  : 0.2660 
Epoch 10 of 50 | Learning rate 1.000e-03
Training Loss: 2.6930e-01 | Validation Loss: 2.6870e-01
Training CC: 0.2795   Validation CC  : 0.3019 
Epoch 11 of 50 | Learning rate 1.000e-03
Training Loss: 2.6334e-01 | Validation Loss: 2.6407e-01
Training CC: 0.3128   Validation CC  : 0.3350 
Epoch 12 of 50 | Learning rate 1.000e-03
Training Loss: 2.6297e-01 | Validation Loss: 2.5985e-01
Training CC: 0.3496   Validation CC  : 0.3658 
Epoch 13 of 50 | Learning rate 1.000e-03
Training Loss: 2.5680e-01 | Validation Loss: 2.5579e-01
Training CC: 0.3781   Validation CC  : 0.3948 
Epoch 14 of 50 | Learning rate 1.000e-03
Training Loss: 2.5061e-01 | Validation Loss: 2.5197e-01
Training CC: 0.4058   Validation CC  : 0.

Epoch 15 of 50 | Learning rate 1.000e-03
Training Loss: 2.3393e-01 | Validation Loss: 2.3310e-01
Training CC: 0.5321   Validation CC  : 0.5425 
Epoch 16 of 50 | Learning rate 1.000e-03
Training Loss: 2.3085e-01 | Validation Loss: 2.3023e-01
Training CC: 0.5568   Validation CC  : 0.5626 
Epoch 17 of 50 | Learning rate 1.000e-03
Training Loss: 2.2956e-01 | Validation Loss: 2.2736e-01
Training CC: 0.5729   Validation CC  : 0.5807 
Epoch 18 of 50 | Learning rate 1.000e-03
Training Loss: 2.2553e-01 | Validation Loss: 2.2444e-01
Training CC: 0.5908   Validation CC  : 0.5999 
Epoch 19 of 50 | Learning rate 1.000e-03
Training Loss: 2.2158e-01 | Validation Loss: 2.2156e-01
Training CC: 0.6107   Validation CC  : 0.6176 
Epoch 20 of 50 | Learning rate 1.000e-03
Training Loss: 2.1820e-01 | Validation Loss: 2.1851e-01
Training CC: 0.6310   Validation CC  : 0.6353 
Epoch 21 of 50 | Learning rate 1.000e-03
Training Loss: 2.1569e-01 | Validation Loss: 2.1539e-01
Training CC: 0.6454   Validation CC  : 

In [108]:
with torch.no_grad():
    all_results = []
    for autoencoder in autoencoders:
        tmp = autoencoder.cpu().latent_vector(demo_data.tensors[0])
        all_results.append(tmp)
all_results = np.column_stack(all_results)

In [109]:
pacmapper = pacmap.PaCMAP(n_components=2, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0)
X = pacmapper.fit_transform(all_results)

In [110]:
imgs = demo_data.tensors[0].numpy()[:,0,...]

In [111]:
label_names = {"Disc":0, "Triangle": 1, "Rectangle":2, "Annulus":3}
assigned_labels = np.zeros(imgs.shape[0])-1
app = latexp.app.build_explorer(images = imgs, 
                                latent_vectors = X, 
                                clusters = np.zeros(imgs.shape[0]), 
                                label_names = label_names,
                                assigned_labels = assigned_labels) 


In [124]:
app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



In [113]:
print(assigned_labels)

[0. 3. 1. ... 1. 2. 2.]


In [131]:
torch.save(demo_data, "ShapesDemo.pt")
np.save("DemoLabels.npy", assigned_labels[0:500])

In [134]:
np.savez_compressed("Demoshapes.npz",demo_data.tensors[0].numpy()[:500,0,...])

In [115]:
for ii,ae in enumerate(autoencoders):
    ae.save_network_parameters("autoencoder_%i.pt"%ii)

In [117]:
for ii in range(3):
    tmp = sparsenet.SparseAutoEncoder_from_file("autoencoder_%i.pt"%ii)

In [133]:
np.save("pacmacX.npy", X[0:500])

In [122]:
import torch
from sklearn.cluster import DBSCAN



obj = DBSCAN(eps=0.750, min_samples=1, leaf_size=5)
clusters = obj.fit_predict(X)

In [123]:
app = latexp.app.build_explorer(images = imgs, 
                                latent_vectors = X, 
                                clusters = clusters, 
                                label_names = label_names,
                                assigned_labels = assigned_labels) 
